# Tarea 2

### Cuerpo Docente

- Profesores: [Andrés Abeliuk](https://aabeliuk.github.io/), [Fabián Villena](https://villena.cl/).
- Profesor Auxiliar: Martín Paredes

### Instrucciones generales

- Grupos de máximo 4 personas.
- Esta prohibido compartir las respuestas con otros grupos.
- Indicios de copia serán penalizados con la nota mínima.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente, si utiliza material extra debe citarlo.


### Integrantes

> POR FAVOR AGREGAR TODOS LOS NOMBRES DE LOS INTEGRANTES

## Contexto

El discurso de odio es cualquier expresión que promueva o incite a la discriminación, la hostilidad o la violencia hacia una persona o grupo de personas en una relación asimétrica de poder, tal como la raza, la etnia, el género, la orientación sexual, la religión, la nacionalidad, una discapacidad u otra característica similar.

En cambio, la incivilidad se refiere a cualquier comportamiento o actitud que rompe las normas de respeto, cortesía y consideración en la interacción entre personas. Esta puede manifestarse de diversas formas, tal como insultos, ataques personales, sarcasmo, desprecio, entre otras.

En esta tarea tendrán a su disposición un dataset de textos con las etiquetas `odio`, `incivilidad` o `normal`. La mayor parte de los datos se encuentra en español de Chile. Con estos datos, deberán entrenar un modelo que sea capaz de predecir la etiqueta de un texto dado.

El corpus para esta tarea se compone de 3 datasets:  
- [Multilingual Resources for Offensive Language Detection de Arango et al. (2022)](https://aclanthology.org/2022.woah-1.pdf#page=136)
- [Dataton UTFSM No To Hate (2022)](http://dataton.inf.utfsm.cl/)
- Datos generados usando la [API de GPT3 (modelo DaVinci 03)](https://platform.openai.com/docs/models/gpt-3).

Agradecimientos a los autores por compartir los datos y a David Miranda, Fabián Diaz, Santiago Maass y Jorge Ortiz por revisar y reetiquetar los datos en el contexto del curso "Taller de Desarrollo de Proyectos de IA" (CC6409), Departamento de Ciencias de la Computación, Universidad de Chile.

Los datos solo pueden ser usados con fines de investigación y docencia. Está prohibida la difusión externa.


## Tarea a resolver

Para esta tarea 2, buscaremos desarrollar un *benchmark* sobre una tarea de clasificación de NLP. Un benchmark es básicamente utilizar diferentes técnicas para resolver una misma tarea específica, en este caso seguiremos buscando alternativas para resolver el problema de clasificación de la tarea 1. Particularmente, se le pide:

- Implementar una arquitectura en RNN utilizando PyTorch.
- Utilizar transformers para revolver el problema de clasificación, en especifico utilizar BETO.
- Utilizar algún LLM utilizando Zero y Few short learning para resolver el problema de clasificación.


### Cargar el dataset


En esta sección, cargaremos el dataset desde el repositorio del módulo. Para ello ejecute las siguientes líneas:

In [2]:
import pandas as pd

In [3]:
# Dataset.
dataset_df = pd.read_csv("https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv", sep="\t")


### Analizar los datos

En esta sección analizaremos el balance de los datos. Para ello se imprime la cantidad de tweets de cada dataset agrupados por la intensidad de sentimiento.

In [16]:
dataset_df.sample(5)

,id,texto,clase
6787,3806,Voy a empezar las clases y los weones de la co...,incivilidad
8062,14113,Decir algo super hiriente sobre las personas t...,odio
4963,5519,LLORONES CTM QUE DE ALIMENTAB DE PENALES JUEGU...,incivilidad
6127,10301,Lo único q quiero es cambio de gobierno wn par...,odio
10354,12019,@user Hijo de puta xD,incivilidad


In [4]:
dataset_df["clase"].value_counts()

,count
clase,
incivilidad,5424
normal,4280
odio,2510


In [17]:
target_classes = list(dataset_df['clase'].unique())
target_classes

['normal', 'incivilidad', 'odio']

### Instalar librerias

Puede usar esta celda para instalar las librerías que estime necesario.

In [18]:
%%capture
from torchtext.data import get_tokenizer



OSError: Could not load this library: /usr/local/lib/python3.12/dist-packages/torchtext/lib/libtorchtext.so

### Importar librerías

En esta sección, importamos la liberías necesarias para el correcto desarrollo de esta tarea. Puede utilizar otras librerías que no se en encuentran aquí, pero debe citar su fuente.

In [7]:
!pip uninstall -y torch torchvision torchaudio

!pip install torch==2.3.0 --index-url https://download.pytorch.org/whl/cpu
!pip uninstall -y scipy
!pip install scipy==1.11.4
!pip install torchtext
!pip install scikit-plot

#falta este
!pip install gensim



Found existing installation: torch 2.6.0+cpu
Uninstalling torch-2.6.0+cpu:
  Successfully uninstalled torch-2.6.0+cpu
Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-2.3.0%2Bcpu-cp312-cp312-linux_x86_64.whl (190.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
timm 1.0.22 requires torchvision, which is not installed.
fastai 2.8.5 requires torchvision>=0.11, which is not installed.


Found existing installation: scipy 1.11.4
Uninstalling scipy-1.11.4:
  Successfully uninstalled scipy-1.11.4
  Using cached scipy-1.11.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached scipy-1.11.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (35.8 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.8.5 requires torchvision>=0.11, which is not installed.
tsfresh 0.21.1 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.11.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires scipy>=1.13, but you have scipy 1.11.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0

In [7]:
import nltk
import numpy as np

from nltk import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin

# importe aquí sus clasificadores

import matplotlib.pyplot as plt

# word2vec
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.phrases import Phrases, Phraser

# Pytorch imports
import torch
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset

from torch import nn
from torch.nn import functional as F

from tqdm import tqdm
from sklearn.metrics import accuracy_score
import gc

from torch.optim import Adam


/usr/local/lib/python3.12/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.12/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


### Crear un clasificador basado en RNN

En esta parte de le pide definir un clasificador utilizando `PyTorch` con alguna arquitectura en Redes Recurrentes. Para ello debe realizar todos los pasos vistos en el tutorial 5, por lo que se recomienda revisarlo. Importante, no puede replicar ningún ejemplo de los del tutorial 5, debe proponer sus propias arquitecturas. Se le recomienda leer como utilizar variaciones de la RNN, como la LSTM o GRU en `Pytorch`.

Para completa esta parte deberá replicar el flujo de trabajo de como utilizar `PyTorch`. Esta esctrictamente prohibido utilizar variaciones que resuelvan directamente este problema, como `PyTorch Lightning` o `TensorFlow`. Los pasos a completar son los siguientes:

[texto del vínculo](https://)#### Cargar el dataset.

In [4]:
# Dataset.
dataset_df = pd.read_csv("https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv", sep="\t")
codes, unique_labels = pd.factorize(dataset_df['clase'])
dataset_df['clase_id'] = codes + 1
dataset_df.head()

,id,texto,clase,clase_id
0,12632,ultimo choro se 2018 que delicia,normal,1
1,7451,Pero es una realidad para muchas mujeres en Ve...,normal,1
2,4211,MALDITA SEAS COMUNA DE ÑUÑOA https://t.co/yN4E...,incivilidad,2
3,10199,Las tontas de #PautaLibre con el tremendo 🌶🌶 ...,incivilidad,2
4,11597,@user @user @user @user @user Devuelvete y and...,odio,3


In [5]:
!wget https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv
import csv

def load_dataset(dataset):
  with open(dataset, encoding='utf-8') as dataset_file:
    reader = csv.reader(dataset_file, delimiter='\t')
    next(reader)
    for row in reader:
      yield int(row[0]), f'{row[1]} {row[2]}'

next(load_dataset('train.tsv'))

--2025-11-13 02:23:07--  https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2080719 (2.0M) [text/plain]
Saving to: ‘train.tsv’

train.tsv           100%[===================>]   1.98M  --.-KB/s    in 0.04s   

2025-11-13 02:23:07 (47.6 MB/s) - ‘train.tsv’ saved [2080719/2080719]



(12632, 'ultimo choro se 2018 que delicia normal')

In [10]:
target_classes

['normal', 'incivilidad', 'odio']

In [6]:
from sklearn.model_selection import train_test_split

feature = dataset_df["texto"]
label = dataset_df["clase_id"]
feature_train, feature_test, label_train, label_test = train_test_split(feature, label)

X_train = feature_train
y_train = label_train

X_test = feature_test
y_test = label_test

train_dataset = [];
test_dataset = [];

train_dataset = list(zip(y_train, X_train))
test_dataset = list(zip(y_test, X_test))

print("IMPORTANTE train_dataset: ")
print(train_dataset)
print("FIN FIN FIN train_dataset: ")

len(train_dataset)




IMPORTANTE train_dataset: 
[(1, 'Necesito marcar los episodios maldita sea https://t.co/c2qZ2Eb0rv'), (2, '@user @user Es harto bruta esta mujer.'), (1, '@user La verdad no entiendo como el Nuevo Liberalismo se fue con Rodolfo, cuando representa todo menos los principios liberales. Cómo es posible que apoye a un tipo que denigra de la mujer y admira a Adolf Hitler.'), (1, 'La Facultad @user participó en el Consejo Asesor Empresarial del Liceo Bicentenario Técnico Las Nieves, el cual destaca por ser un colegio técnico de mujeres con la carrera de telecomunicaciones 👏 https://t.co/79Siq2Jhjl'), (2, 'NO TE CREO MARIPAN CTM'), (2, '@user @user Me importa MEDIO KM DE PICO LO QUE DIGA SOLABARRIETA Y TUDOR...\nNo tienen idea de fútbol. Sobretodo el primero'), (3, '@user @user @user Otro Qlio que le gusta la Wea!    🤦'), (2, '@user puta vida'), (3, '@user @user @user Confundes tu emoción o visión ideológica con un derecho superior del niño.  Que dos personas homo se amen no les da derecho a qu

9160

#### Definir el vocabulario.

In [12]:
import re

def get_tokens(texto):

  for frase in texto:
        clean_text = re.sub(r'[^A-Za-zñáéíóú]', ' ', frase)
        tokens = clean_text.split()

  return tokens

#### Cargar el `DataLoader`

Recuerde que podría necesitar una función intermedia para procesar cada batch durante el entrenamiento, pero no es obligatorio hacerlo.

In [13]:
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [14]:


def build_vocabulary(datasets):
    for dataset in datasets:
        for _, text in dataset:
            yield get_tokens(text)

vocab = build_vocab_from_iterator(build_vocabulary([train_dataset, test_dataset]), min_freq=1, specials=["<UNK>"])

vocab.set_default_index(vocab["<UNK>"])

In [15]:
##################################################
train_dataset, test_dataset  = to_map_style_dataset(train_dataset), to_map_style_dataset(test_dataset)

#target_classes = ["World", "Sports", "Business", "Sci/Tech"]

max_words = 25
##################################################
def vectorize_batch(batch):
    Y, X = list(zip(*batch))
    X = [vocab(get_tokens(text)) for text in X]
    X = [tokens+([0]* (max_words-len(tokens))) if len(tokens)<max_words else tokens[:max_words] for tokens in X] ## Bringing all samples to max_words length.

    return torch.tensor(X, dtype=torch.int32), torch.tensor(Y) - 1 ## We have deducted 1 from target names to get them in range [0,1,2,3] from [1,2,3,4]
##################################################


train_loader = DataLoader(train_dataset, batch_size=1024, collate_fn=vectorize_batch, shuffle=True)
test_loader  = DataLoader(test_dataset , batch_size=1024, collate_fn=vectorize_batch)

#### Definir la red recurrente.

Recuerde que debe difirnir los hyperparametros que estime conveniente.

In [16]:
##################################################
import torch
import torch.nn as nn

# Suponiendo que estas variables están definidas en tu contexto:
# vocab_size = 10000
embed_len = 128
hidden_dim = 256
n_layers = 2
num_classes = len(dataset_df["clase"].unique())

class RNNClassifier(nn.Module):
    def __init__(self):
        super(RNNClassifier, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=embed_len)
        self.lstm = nn.LSTM(input_size=embed_len, hidden_size=hidden_dim, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, num_classes)
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

    def forward(self, X_batch):
        # X_batch shape: (batch_size, sequence_length)
        embeddings = self.embedding_layer(X_batch)
        # embeddings shape: (batch_size, sequence_length, embed_len)

        # Inicializar el estado oculto y el estado de la celda
        h0 = torch.zeros(self.n_layers, X_batch.size(0), self.hidden_dim).to(X_batch.device)
        c0 = torch.zeros(self.n_layers, X_batch.size(0), self.hidden_dim).to(X_batch.device)

        # Pasar los embeddings a la capa LSTM
        output, (hn, cn) = self.lstm(embeddings, (h0, c0))
        # output shape: (batch_size, sequence_length, hidden_dim)
        # hn shape: (n_layers, batch_size, hidden_dim) - hidden state for the last time step
        # cn shape: (n_layers, batch_size, hidden_dim) - cell state for the last time step

        # Tomamos el estado oculto de la última capa y del último paso de tiempo para la clasificación
        # hn[-1] corresponde al estado oculto de la última capa
        return self.linear(hn[-1])


#print(num_classes)


#### Funciones de entrenamiento y evaluación.

Para esta parte, puede utilizar las funciones vista en el tutorial directamente. Pero es su reponsabilidad ajustarlas a su código.

In [17]:
def CalcValLossAndAccuracy(model, loss_fn, val_loader):
    with torch.no_grad():
        Y_shuffled, Y_preds, losses = [],[],[]
        for X, Y in val_loader:
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())

            Y_shuffled.append(Y)
            Y_preds.append(preds.argmax(dim=-1))

        Y_shuffled = torch.cat(Y_shuffled)
        Y_preds = torch.cat(Y_preds)

        print("Valid Loss : {:.3f}".format(torch.tensor(losses).mean()))
        print("Valid Acc  : {:.3f}".format(accuracy_score(Y_shuffled.detach().numpy(), Y_preds.detach().numpy())))


def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=10):
    for i in range(1, epochs+1):
        losses = []
        for X, Y in tqdm(train_loader):
            Y_preds = model(X)

            loss = loss_fn(Y_preds, Y)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("Train Loss : {:.3f}".format(torch.tensor(losses).mean()))
        CalcValLossAndAccuracy(model, loss_fn, val_loader)

def MakePredictions(model, loader):
    Y_shuffled, Y_preds = [], []
    for X, Y in loader:
        preds = model(X)
        Y_preds.append(preds)
        Y_shuffled.append(Y)
    gc.collect()
    Y_preds, Y_shuffled = torch.cat(Y_preds), torch.cat(Y_shuffled)

    return Y_shuffled.detach().numpy(), F.softmax(Y_preds, dim=-1).argmax(dim=-1).detach().numpy()



##### Entrenamiento del modelo

Ejecute el entrenamiento de su modelo propuesto.

In [18]:
#train_loader = feature_train
#test_loader = feature_test


epochs = 1
learning_rate = 1e-3

loss_fn = nn.CrossEntropyLoss()
rnn_classifier = RNNClassifier()
optimizer = Adam(rnn_classifier.parameters(), lr=learning_rate)

TrainModel(rnn_classifier, loss_fn, optimizer, train_loader, test_loader, epochs)

#feature_train, feature_test, label_train, label_test = train_test_split(feature, label)


100%|██████████| 9/9 [00:20<00:00,  2.30s/it]


Train Loss : 1.063
Valid Loss : 1.051
Valid Acc  : 0.458


##### Evaluacion del modelo

Ejecute la evaluación de su modelo, y genere un reporte de evaluación similar al de la pregunta anterior.

In [19]:
target_classes = ["normal", "incivilidad", "odio"]

In [20]:
Y_actual, Y_preds = MakePredictions(rnn_classifier, test_loader)

In [21]:
print("Test Accuracy : {}".format(accuracy_score(Y_actual, Y_preds)))
print("\nClassification Report : ")
print(classification_report(Y_actual, Y_preds, target_names=target_classes))
print("\nConfusion Matrix : ")
print(confusion_matrix(Y_actual, Y_preds))

Test Accuracy : 0.4577603143418468

Classification Report : 
              precision    recall  f1-score   support

      normal       0.00      0.00      0.00      1042
 incivilidad       0.46      1.00      0.63      1398
        odio       0.00      0.00      0.00       614

    accuracy                           0.46      3054
   macro avg       0.15      0.33      0.21      3054
weighted avg       0.21      0.46      0.29      3054


Confusion Matrix : 
[[   0 1042    0]
 [   0 1398    0]
 [   0  614    0]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Finalmente, análice sus resultados, ¿Porqué cree que obtuvo esos resultados?, ¿Es mejor que sólo utilizar Word Embeddings, porque?. Justique.

### Transformers BERT.

Para esta tarea se le piden crear una representación de texto usando la arquitectura basada en transformers, BERT:



In [22]:
!pip install transformers


#### Import BETO

Para esto debe importar el tokenizador y el modelo BETO.

In [8]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

#### Ejemplo de extracción de features.

In [25]:
# Uninstall the older version and then install the required modern version
!pip uninstall torch -y
!pip install --upgrade torch torchvision torchaudio

Found existing installation: torch 2.3.0+cpu
Uninstalling torch-2.3.0+cpu:
  Successfully uninstalled torch-2.3.0+cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 122.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 797.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 834.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 866.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 MB 762.2 kB/s eta 0:00:00
  

Luego, debe cargar los modelos pre-entrenados:

In [35]:
tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
model = AutoModelForMaskedLM.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', output_hidden_states=True)

Una vez cargado BETO, debe utilizarlo para extraer los embeddings asociados a la texto de su corpus. Se espera que usted realice lo siguiente:



Tokenizamos el texto para extraer los ids a cada palabra en el vocabulario interno de BETO, se recomienda utilizar el padding, trunctation, y max_length para considerar oraciones de diferentes tamaños.

Luego, debe verificar si cada uno de los ids extraídos se encuentran en la misma máquina donde fue cargado el modelo (CPU o GPU), se recomienda dejar todo en GPU.

In [36]:
# oración
sentence = "Hola, que tal? me gusta mucho el curso de NLP"

# extraemos los ids de los tokens, se recomienda definir los valores de las variables:
#  padding, truncation, max_length debido a que las secuencia de texto puede tener diferentes largos
inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt", max_length=512)
# revisamos si cada de los ids, se encuentran en la misma máquina que el modelo (GPU o CPU)
inputs = {k: v.to(model.device) for k, v in inputs.items()}
inputs

{'input_ids': tensor([[    4,  1734,  1019,  1041,  1713,  1059,  1094,  2331,  1700,  1039,
           4430,  1009, 15567, 30968,     5]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

Una vez, extraídos los ids, usted debe obtener los estados ocultos de las ultimas capas de BETO.

In [37]:
# Extraemos la features
outputs = model(**inputs)

# ahora `outputs` tendrá el atributo `hidden_states`
hidden_states = outputs.hidden_states


Finalmente, debe guardar los embeddings en CPU los embeddings del token [CLS] que será usados en la clasificación del análisis de sentimientos.

In [38]:
import torch

In [39]:
with torch.no_grad():
# Seleccionamos la última capa y obtenemos el primer token ([CLS]) para cada ejemplo
# Estos son los embeddings que normalmente se usan para tareas de clasificación.
  cls_embeddings = hidden_states[-1][:, 0, :].cpu().numpy()


#### Extraer features de todo el dataset

Considerando lo anterior, usted debe implementar la función `get_beto_features_in_batches` para extraer los features de BETO (los estados ocultos y los embeddings del token [CLS]).

Esta función recibe dos parámetros, el texto a vectorizar y un tamaño de batch, debido a que es extramadamente recomendable a que procesen el texto por lotes, ya que si cargan todos el modelo se les agotará la memoria RAM.

In [40]:
from datasets import load_dataset, Dataset, DatasetDict

In [41]:
train_dataset = Dataset.from_dict({
    'text': feature_train.tolist(),
    'label': label_train.tolist()
})

validation_dataset = Dataset.from_dict({
    'text': feature_train.tolist(),
    'label': label_train.tolist()
})

# Create the test Dataset
test_dataset = Dataset.from_dict({
    'text': feature_test.tolist(),
    'label': label_test.tolist()
})

# --- 4. Combine into the DatasetDict ---
my_dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset,
})

In [29]:
my_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9160
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 9160
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3054
    })
})

In [34]:
'''from transformers import AutoTokenizer
from transformers import AutoModel
from datasets import load_dataset
import torch

# Changed model_ckpt to BETO for consistency
model_ckpt = "dccuchile/bert-base-spanish-wwm-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_beto = AutoModel.from_pretrained(model_ckpt).to(device)'''

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:

def tokenize(batch):
  # Por cada batch vamos a obtener el texto, tokenizarlo y aplicar padding
  return tokenizer(batch["text"], padding=True, truncation=True, max_length=256)

def extract_hidden_states(batch):
 # Vamos a extraer los hidden states desde BERT, para esto pasamos el modelo
 # dispositivo
 inputs = {k:v.to(device) for k,v in batch.items() if k in tokenizer.model_input_names}
 # Extraemos solamente la last hidden state
 with torch.no_grad():
   # Changed model_distilbert to model_beto
   last_hidden_state = model_beto(**inputs).last_hidden_state
 # representaciones vectoriales que tanto queriamos :D
 return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

In [51]:
# Función para procesar los textos en lotes y obtener las características de BETO

BATCH_SIZE = 16



def get_beto_features_in_batches(texts, batch_size):
  texts_encoded = texts.map(tokenize, batched=True, batch_size=BATCH_SIZE)
  texts_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label", "text"])
  texts_hidden = texts_encoded.map(extract_hidden_states, batched=True, batch_size=BATCH_SIZE)
  return texts_hidden

device


device(type='cuda')

In [52]:
train_embs = get_beto_features_in_batches(my_dataset_dict, BATCH_SIZE)

Map:   0%|          | 0/9160 [00:00<?, ? examples/s]

Map:   0%|          | 0/9160 [00:00<?, ? examples/s]

Map:   0%|          | 0/3054 [00:00<?, ? examples/s]

Map:   0%|          | 0/9160 [00:00<?, ? examples/s]

Map:   0%|          | 0/9160 [00:00<?, ? examples/s]

Map:   0%|          | 0/3054 [00:00<?, ? examples/s]

Ahora extraíga los features, un punto importante es que la extracción de features podría tomar alrededor de una a dos horas dependiendo del tamaño del batch que utilicen.

In [ ]:
train_embs = get_beto_features_in_batches(..., ...)

In [53]:
type(train_embs)

datasets.dataset_dict.DatasetDict

In [54]:
train_embs

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'hidden_state'],
        num_rows: 9160
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'hidden_state'],
        num_rows: 9160
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'hidden_state'],
        num_rows: 3054
    })
})

#### Clasificación

Una vez extraído los features de BETO, realice la clasificación de los embeddings obtenidos. Debe implementar dos clasificadores a su elección.

In [55]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# Assuming your DatasetDict is named 'bert_dataset_dict'
# Replace 'bert_dataset_dict' with the actual name of your dictionary.

# Extract the features (X) and labels (y) for the training set
X_train = np.array(train_embs['train']['hidden_state'])
y_train = np.array(train_embs['train']['label'])

# Extract the features (X) and labels (y) for the test set
X_test = np.array(train_embs['test']['hidden_state'])
y_test = np.array(train_embs['test']['label'])

print(f"Train Features Shape: {X_train.shape}")
print(f"Test Features Shape: {X_test.shape}")

Train Features Shape: (9160, 768)
Test Features Shape: (3054, 768)


In [56]:
# Create the new Pipeline
clf1 = Pipeline([
    # Step 1: Normalize or scale the dense embedding features
    ('scaler', StandardScaler()),

    # Step 2: The classifier that takes the numerical data
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))
])

In [57]:
# Train the classifier
print("Training the RandomForestClassifier...")
clf1.fit(X_train, y_train)
print("Training complete.")

# Evaluate the classifier
from sklearn.metrics import classification_report

y_pred = clf1.predict(X_test)
report = classification_report(y_test, y_pred)

print("\n--- Classification Report (Test Set) ---")
print(report)

Training the RandomForestClassifier...
Training complete.

--- Classification Report (Test Set) ---
              precision    recall  f1-score   support

           1       0.61      0.59      0.60      1032
           2       0.61      0.82      0.70      1405
           3       0.82      0.21      0.33       617

    accuracy                           0.62      3054
   macro avg       0.68      0.54      0.54      3054
weighted avg       0.65      0.62      0.59      3054



#### Reporte de evaluación

Una vez realizada la clasificación, realice el reporte de clasificación y el análsis de la matriz confusión para ambos clasificadores.

Recuerde que para hacer esto, debe extraer los features del dataset de testing.

Finalmente, que puede decir de los resultados obtenidos. ¿Se diferencia de los resultados obtenidos de la red RNN? ¿A que se debe esto? Justifique

### Large Language Model

##### Zero Shot Learning

Utilizando la técnica de zero shot learning, utilice la API de `openai` para clasificar el texto del dataset.

Además, genere el reporte de clasificación usando `scikit-learn` como en las preguntas anteriores.

Recuerde solicitar al profesor auxiliar el TOKEN para hacer consultas al LLM.

##### Few Shot Learning

Utilizando la técnica de few shot learning, utilice la API de `openai` para clasificar el texto del dataset.

Además, genere el reporte de clasificación usando `scikit-learn` como en las preguntas anteriores.

Recuerde solicitar al profesor auxiliar el TOKEN para hacer consultas al LLM.